<a href="https://colab.research.google.com/github/Felgaba/HSE_DS_MasterTrack/blob/main/UZB_DEPOSITS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [ ]:
banks_list = []
deposits_list = []
interests_list = []
maturities_list = []
off_online_list = []
match_interest = [] #list to fill with the regex values

for j in range(1, 10):
  url = "https://bank.uz/deposits/valyutnye-vklady?arrFilter_22_MAX=0&SROK=&set_filter=yes&PAGEN_1=" + str(j)
  r = requests.get(url)
  soup = BeautifulSoup(r.content)
  
  # general info
  bank_name = soup.find_all('span', attrs={'class': 'medium-text'})
  
  #banks_name
  bank_names = soup.find_all('div', attrs={'class': 'table-card-offers-block1-text'})
  for i in range(len(bank_names)): 
    banks_list.append(bank_names[i].find('span', attrs={'class': 'medium-text'}).text)
    deposits_list.append(bank_names[i].find('a', attrs={'target': '_blank'}).text) 

  #interest, maturity, on_offline
  deposits_data = soup.find_all('div', attrs={'class': 'table-card-offers-block-all'})
  interests_list = []
  for i in range(len(deposits_data)): 
    interests_list.append(deposits_data[i].find('span', attrs={'class': 'medium-text'}).text)
  regex = '[\d+\.*\,*\d*]+'
  for i in range(len(interests_list)):
    v = re.findall(regex, interests_list[i])
    if v == []:
      match_interest.append("Не указано")
    else:
      match_interest.append(max(v))

  
  maturity = soup.find_all('div', attrs={'class': 'table-card-offers-block3'})
  for i in range(len(maturity)):
    maturities_list.append(maturity[i].find('span', attrs={'class': 'medium-text'}).text)
  maturities_list = [re.sub('\n', '', i) for i in maturities_list]
  
  
  off_online = soup.find_all('div', attrs={'class': 'table-card-offers-block5'})
  for i in range(len(off_online)):
    x = off_online[i].find_all('span', attrs={'class': 'medium-text'})
    #print(x, len(x))
    if x == []:
      off_online_list.append('Не указано')
    elif len(x) == 2:
      off_online_list.append('Оба')
    else:
      off_online_list.append(x[0].text)

In [ ]:
dct = {'Банк': banks_list, 'Название вклада': deposits_list, "Ставка": match_interest, "Продолжительность": maturities_list, "Как открыть?": off_online_list}
deposits = pd.DataFrame.from_dict(dct)

In [ ]:
deposits.to_excel("uzb_deposits.xlsx")

In [ ]:
deposits[(deposits['Банк'] == 'Асакабанк') | (deposits['Банк'] == 'Ипак Йули Банк') ]

,Банк,Название вклада,Ставка,Продолжительность,Как открыть?
1,Асакабанк,Аванс (usd),2,6 месяцев - 1 год,Оба
10,Асакабанк,On-line (usd),2.5,1 год 1 месяц,Оба
20,Асакабанк,Максимальная выгода (евро),1.8,1 месяц - 2 года,Оба
26,Асакабанк,Максимальная выгода (usd),2.8,1 месяц - 2 года,Оба
34,Ипак Йули Банк,Обод турмуш-5,3,3 месяца,Банк
38,Ипак Йули Банк,Бахор Тикланиш - 8,3,6 месяцев,Банк
40,Асакабанк,Аванс (евро),1.2,6 месяцев - 1 год,Банк
41,Асакабанк,On-line (евро),1.6,1 год 1 месяц,Оба
45,Ипак Йули Банк,Бахор Тикланиш - 9,3,1 год,Не указано
